In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt
import pandas as pd
from src.model import BasicModel

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False, matching_type="general", jump=3):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
        self.matching = matching_type
        self.jump = jump
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    ['week sin',
    'week cos'],
    ['year sin',
    'year cos'],
    'season idx',
    'weather idx',
    'avg ta',
    'avg rhm'
]

In [5]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
#     "multivariate (year)": [
#         'energy (kw 15min)',
#         'year sin',
#         'year cos',
#     ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
#     "multivariate (avg ta)": [
#         'energy (kw 15min)',
#         'avg ta'
#     ],
#     "multivariate (avg rhm)": [
#         'energy (kw 15min)',
#         'avg rhm'
#     ],
#     "multivariate (season)": [
#         'energy (kw 15min)',
#         'season idx'
#     ]
}

# Input Report Simulation
IRS = []

for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )
    
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch,
                   matching_type=_.matching,
                   is_multi_step=True,
                   model_type="Multi Step / Autoregressive")
    
    bm.set_window(OUT_STEPS=3)
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_multi_predict(is_reshape=True)
    bm.set_multi_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (weather)) set predict (validation) info start ###### 

0 / 99
50 / 99
99 / 99 complete.

###### [Notice] set predict info success ###### 



In [6]:
test_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.870581,53.587466,0.338774,0.008454,1.953239,0.860492,0.208452,1244.984038,577.828646
multivariate (week) / with cluster,0.876794,55.628758,0.325705,0.008962,1.872649,0.872258,0.199284,1244.984038,552.414875
multivariate (weather) / with cluster,0.873532,52.607518,0.332999,0.005968,1.930637,0.875006,0.212853,1244.984038,590.028841
univariate,0.881063,58.588539,0.309081,0.004559,1.847230,0.874385,0.185991,1244.984038,515.566082
multivariate (week),0.892276,62.185077,0.297105,0.008004,1.738594,0.883811,0.169838,1244.984038,470.789758
multivariate (weather),0.883537,59.549290,0.307215,0.009669,1.827618,0.876688,0.181676,1244.984038,503.604886


In [7]:
val_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.911034,66.824822,0.263987,0.008928,1.467371,0.885585,0.110781,694.234627,230.313570
multivariate (week) / with cluster,0.905005,64.866164,0.272716,0.005310,1.492253,0.877640,0.117321,694.234627,243.911257
multivariate (weather) / with cluster,0.911261,67.622902,0.248383,0.010512,1.453260,0.885489,0.108116,694.234627,224.773028
univariate,0.912427,66.982036,0.243671,0.008249,1.450469,0.890861,0.110256,694.234627,229.222137
multivariate (week),0.912469,67.535681,0.240162,0.000959,1.412994,0.889834,0.108407,694.234627,225.378543
multivariate (weather),0.913573,67.504060,0.240585,0.010128,1.434960,0.891883,0.108513,694.234627,225.598065
